In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict

stats = pd.read_csv('CHARTEVENTS.csv')
stats = stats.drop(columns = ['ROW_ID', 'ICUSTAY_ID', 'STORETIME', 'CGID', 'WARNING', 'ERROR',
                              'RESULTSTATUS', 'STOPPED'])
stats = stats.dropna(axis=0)
items = pd.read_csv('D_ITEMS.csv')
items = items[['ITEMID', 'LABEL']]
chart = pd.merge(stats, items, on='ITEMID', how='inner')
new_data = pd.get_dummies(chart, columns=['LABEL'])

new_data.columns = new_data.columns.str.replace(' ', '_')
new_data.rename(columns={'LABEL_Heart_Rate':'heart_rate','LABEL_Non_Invasive_Blood_Pressure_systolic':'systolic',
                        'LABEL_Non_Invasive_Blood_Pressure_diastolic':'diastolic',
                        'LABEL_O2_saturation_pulseoxymetry':'SpO2', 'LABEL_Respiratory_Rate_(Set)':'RR',
                        'LABEL_Temperature_Fahrenheit':'temp_f', 'LABEL_Temperature_Celsius':'temp_c'}, inplace=True)
chart = new_data[['SUBJECT_ID','HADM_ID','ITEMID','CHARTTIME','VALUE','VALUENUM','VALUEUOM','heart_rate','systolic','diastolic','SpO2','RR','temp_f','temp_c']]

pd.options.mode.chained_assignment = None
chart.heart_rate.astype(int)
#type(chart.heart_rate[3] * chart.VALUE[3])
#type(chart.heart_rate[3])
#type(chart.heart_rate)
chart['HEART_RATE'] = chart.heart_rate * chart.VALUENUM
chart['SYSTOLIC'] = chart.systolic * chart.VALUENUM
chart['DIASTOLIC'] = chart.diastolic * chart.VALUENUM
chart['SPO2'] = chart.SpO2 * chart.VALUENUM
chart['RES_PRATE'] = chart.RR * chart.VALUENUM
chart['temp_f_tell'] = chart.temp_c.replace({0:np.nan})
#chart.temp_c.apply(lambda x: (1 if x == 1 else ))
chart['temp_f_con'] = (chart.temp_f_tell * 2.8) + 32
chart['TEMP'] = (chart.temp_f * chart.VALUENUM) + (chart.temp_f_con)
chart['TEMPERATURE'] = chart.TEMP.replace({np.nan:0})
chart = chart.drop(columns = ['SUBJECT_ID','VALUE','VALUENUM','heart_rate','systolic','diastolic','SpO2','RR','temp_f','temp_c','temp_f_tell','temp_f_con','TEMP'])
chart = chart.replace(0,np.nan)
chart = chart.dropna(axis=0,thresh=2)
chart = chart.set_index('HADM_ID')
chart.head(50)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# used to check labels in LABELS
#d = defaultdict(int)
#for i in chart.LABEL:
#    d[i] += 1
#d